In [ ]:
import multiprocessing
from multiprocessing import Pool
import pandas as pd
from PKM_multiprocess import ParallelCompute_PKM as PC_PKM

# Конструкция аккумулятора
constr = [{
    "Diametr": 10,
    "kolichestvo": 8,
    "Visota": 15,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
},{
    "Diametr": 11,
    "kolichestvo": 81,
    "Visota": 151,
    "lambda_min_vata": 0.0451,
    "delta_min_vata": 0.011,
}]

temperatures=[-29,-25, -20, -15, -10, -5, 0, 5, 8]
arguments=[(temperatures[i],
           constr[j]
           )
            
           for i in range(len(temperatures))
           for j in range(len(constr))
        
]
print(arguments)
if __name__ == "__main__":
    pool = Pool(processes=len(arguments))
    Result = pool.map(PC_PKM, arguments)
    # Result = pool.map(PC_PKM, [-29,  -10,  0, 8])
    pool.close()
    pool.join()
    # df = pd.DataFrame(Result)
    # # создаются новые таблицы после каждого расчета
    # for i in range(len(Result)):
    #     nametable = "resdataTemp"+str(df['T_air'][i])+".xlsx"
    #     with pd.ExcelWriter(nametable) as writer:
    #         df.to_excel(writer, sheet_name="all_result", index=False)
            


[(-29, {'Diametr': 10, 'kolichestvo': 8, 'Visota': 15, 'lambda_min_vata': 0.045, 'delta_min_vata': 0.01}), (-29, {'Diametr': 11, 'kolichestvo': 81, 'Visota': 151, 'lambda_min_vata': 0.0451, 'delta_min_vata': 0.011}), (-25, {'Diametr': 10, 'kolichestvo': 8, 'Visota': 15, 'lambda_min_vata': 0.045, 'delta_min_vata': 0.01}), (-25, {'Diametr': 11, 'kolichestvo': 81, 'Visota': 151, 'lambda_min_vata': 0.0451, 'delta_min_vata': 0.011}), (-20, {'Diametr': 10, 'kolichestvo': 8, 'Visota': 15, 'lambda_min_vata': 0.045, 'delta_min_vata': 0.01}), (-20, {'Diametr': 11, 'kolichestvo': 81, 'Visota': 151, 'lambda_min_vata': 0.0451, 'delta_min_vata': 0.011}), (-15, {'Diametr': 10, 'kolichestvo': 8, 'Visota': 15, 'lambda_min_vata': 0.045, 'delta_min_vata': 0.01}), (-15, {'Diametr': 11, 'kolichestvo': 81, 'Visota': 151, 'lambda_min_vata': 0.0451, 'delta_min_vata': 0.011}), (-10, {'Diametr': 10, 'kolichestvo': 8, 'Visota': 15, 'lambda_min_vata': 0.045, 'delta_min_vata': 0.01}), (-10, {'Diametr': 11, 'kolich